# Imports

In [6]:
from datetime import datetime
import firecloud.api as FAPI
import json
import os
import pandas as pd
import pprint

print(FAPI.whoami())
pp = pprint.PrettyPrinter(indent=4)

pet-117272931645288568532@terra-e36fcccd.iam.gserviceaccount.com


In [10]:
BILLING_PROJECT_ID = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

CATALOG_WORKSPACE_NAMESPACE = "anvil-datastorage"

print("Copy files:")
# print(WORKSPACE_BUCKET)
!gsutil cp $WORKSPACE_BUCKET/notebooks/attributes_for_AnVIL_workspaces_update_210217.csv  .
!gsutil cp $WORKSPACE_BUCKET/notebooks/attributes_for_AnVIL_workspaces_220204.csv  .
!gsutil cp $WORKSPACE_BUCKET/notebooks/anvil_workspaces_tags_220211.tsv  .

print("List all files:")
!ls 
# print(BILLING_PROJECT_ID)

def getRequestorPays(bucketName):
    result = !gsutil -u anvil-jamboree-june-10-2020 requesterpays get  gs://$bucketName
    return ( "Enabled" in result[0])

def getBucketSize(bucketName):
    result = !gsutil -u anvil-jamboree-june-10-2020 du -s gs://$bucketName
    return result[0].split()[0]
  

# print(getBucketSize('fc-56ac46ea-efc4-4683-b6d5-6d95bed41c5e'))
    

Copy files:
Copying gs://fc-cb5be780-171f-49d4-9116-b77fd2237d0b/notebooks/attributes_for_AnVIL_workspaces_update_210217.csv...
/ [1 files][293.9 KiB/293.9 KiB]                                                
Operation completed over 1 objects/293.9 KiB.                                    
Copying gs://fc-cb5be780-171f-49d4-9116-b77fd2237d0b/notebooks/attributes_for_AnVIL_workspaces_220204.csv...
/ [1 files][709.1 KiB/709.1 KiB]                                                
Operation completed over 1 objects/709.1 KiB.                                    
Copying gs://fc-cb5be780-171f-49d4-9116-b77fd2237d0b/notebooks/anvil_workspaces_tags_220211.tsv...
/ [1 files][105.6 KiB/105.6 KiB]                                                
Operation completed over 1 objects/105.6 KiB.                                    
List all files:
'AnVIL Dataset Catalog Notebook.ipynb'
'AnVIL Dataset Catalog Test.ipynb'
 anvil_workspaces_tags_220211.tsv
 attributes_for_AnVIL_workspaces_220204.csv
 attrib

In [11]:
# dir(FAPI)
# help(FAPI)

In [12]:
## Get all workspaces we have access to
workspaces = FAPI.list_workspaces().json()


workspaceNames = set()
for w in workspaces:
    workspaceNames.add(w['workspace']['name'])
#pp.pprint(workspaces[1])
     

In [13]:
## Read in full workspaces list
attributesSheet = pd.read_csv('attributes_for_AnVIL_workspaces_220204.csv')
attributesSheet = attributesSheet[["name","consortium"]]
attributesSheet.set_index("name", inplace=True)


## Read in tags list
tagsSheet = pd.read_csv('attributes_for_AnVIL_workspaces_update_210217.csv')
tagsSheet = tagsSheet[["name","tag:tags.items", "library:dataUseRestriction"]]
tagsSheet.set_index("name", inplace=True)


# print(workspaces)

## Filter out workpaces in the catalog that we have acces to
noAccess = set()
access = set()

for wName in attributesSheet.index.values:
    if wName not in workspaceNames:
        noAccess.add(wName)
    else:
        access.add(wName)

print("Total catalog workspaces:" , len(access)+len(noAccess))
print("Catalog workspaces in anvil_dev:",len(access))
print("")

print("Catalog Workspaces not in anvil_dev:", len(noAccess))
print("")
print(",\n".join(noAccess))



Total catalog workspaces: 535
Catalog workspaces in anvil_dev: 516

Catalog Workspaces not in anvil_dev: 19

AnVIL_CMG_Broad_Genitourinary_Hirschhorn_WES,
AnVIL_CMG_Broad_Muscle_INMR-Cooper_WGS,
AnVIL_CMG_Broad_Orphan_Chung_WGS,
AnVIL_CCDG_Broad_NP_Epilepsy_DEUUGS_DS_EP_MDS_WES,
AnVIL_CSER_CSERUWDCC,
AnVIL_CMG_UWash_GRU_1,
ANVIL_CMG_Broad_Genitourinary_Sinclair_WES,
AnVIL_CCDG_NHGRI_Broad_ASD_Daly_phs000298_WES_vcf,
AnVIL_CMG_Broad_Kidney_Southampton_WES,
AnVIL_CMG_Broad_Kidney_Southampton_WGS,
AnVIL_CMG_Broad_MonogenicDiabetes_Philipson_WES,
AnVIL_CSER_NYCKIDSEQ,
AnVIL_ccdg_asc_ndd_daly_talkowski_aleksic_asd_exome,
AnVIL_CSER_NCGENES2,
AnVIL_CMG_Broad_Brain_Roscioli_WGS,
AnVIL_ccdg_asc_ndd_daly_talkowski_gargus_asd_exome,
ANVIL_GATK_10X,
AnVIL_CMG_Broad_Rare_RGP_WGS,
AnVIL_ccdg_asc_ndd_daly_talkowski_hultman_asd_exome


In [14]:



catalogArray = []

def extractDataTypes(workspace):
    attributes = w["workspace"]["attributes"]
    dataTypes = attributes.get("library:datatype","Unspecified")
    if isinstance(dataTypes, dict):
        return  ",".join(dataTypes["items"])  # cant use dataType.items as items is a function on dict
    else:
        return dataTypes
    
def extractTag(workspace, prefix):
    attributes = w["workspace"]["attributes"]
    tags = attributes.get("tag:tags","Unspecified")
    if isinstance(tags, dict):
        items = tags["items"]  # cant use dataType.items as items is a function on dict
        for tag in items:
            if tag.startswith(prefix):
                return tag.lstrip(prefix);
        return "Unspecified"
    else:
        return tags

def getSamples(workspace):
    return FAPI.get_entities(CATALOG_WORKSPACE_NAMESPACE, workspace,'sample').json()

def getSubjects(workspace):
    return FAPI.get_entities(CATALOG_WORKSPACE_NAMESPACE, workspace,'subject').json()

def getParticipants(workspace):
    return FAPI.get_entities(CATALOG_WORKSPACE_NAMESPACE, workspace,'participant').json()

def getEntityCount(entities, entity):
    e = entities.get(entity,"Unspecified")
    if isinstance(e, dict):
        return e['count']
    else:
        return 0
    

def getEntityCounts(workspace):
    entities = FAPI.list_entity_types(CATALOG_WORKSPACE_NAMESPACE,workspace).json()
    return {
        'discovery': getEntityCount(entities, "discovery"),
        'family': getEntityCount(entities, "family"),
        'participant': getEntityCount(entities, "participant"),
        'sample': getEntityCount(entities, "sample"),
        'subject': getEntityCount(entities, "subject")

    }

def extractTagSheetTag(workspaceName,tagName):
    if workspaceName in tagsSheet.index:
        return tagsSheet.at[workspaceName,tagName]
    else:
        return "Unspecified"

def extractTagSheetTagItem(workspaceName, tagName):
    if workspaceName in tagsSheet.index:
        items = tagsSheet.at[workspaceName,"tag:tags.items"]
        items = cleanUpDbGapTagSheetTag(items)
        items = items.split(",")
        for item in items:
            item = item.strip()
            if item.startswith(tagName):
                parts = item.split(":")
                if len(parts) == 2:
                    return parts[1]
                else:
                    return "Unspecified"
        ## Not found reuturn unspecified
        return "Unspecified"
    return "Unspecified"
    

def cleanUpDbGapTagSheetTag(tag):
    tag = tag.replace("'","")
    tag = tag.replace("[","")
    tag = tag.replace("]","")
    return tag




In [15]:
# cleanUpDbGapTagSheetTag(extractTagSheetTag("AnVIL_CCDG_Baylor_CVD_EOCAD_BioMe_WGS","tag:tags:dbGaP"))

In [16]:
#  'accessLevel': 'READER'
reader = set()
writer = set()
for w in workspaces:
    name = w['workspace']['name']
    if name in access:
#         print(w['accessLevel'], name )
        if w['accessLevel'] == "READER":
            reader.add(name)
        if w['accessLevel'] == "WRITER" or w['accessLevel'] == "OWNER":
            writer.add(name)

print("Catalog workspaces with reader access:",len(reader))
print("")

print("Catalog Workspaces with writer access:", len(writer))
     
print("Writer workspaces")
# print(writer)

print("Reader workspaces")
# print(reader)  


Catalog workspaces with reader access: 473

Catalog Workspaces with writer access: 43
Writer workspaces
Reader workspaces


In [17]:
wsCount = 1
for w in workspaces:
    name = w['workspace']['name']   
    if name in access:
        print("{wsCount} - {name}".format(wsCount=wsCount, name=name))
        wsCount = wsCount+1
#         if wsCount > 20:
#             break
        attributes = w["workspace"]["attributes"]
        entityCounts = getEntityCounts(name)
        bucketName = w["workspace"]["bucketName"]
        wsVals = {
            'name': name,
            'sheet:consortium': attributesSheet.at[name,"consortium"] ,
             #Extract consortia tag item
            'tag:tags:consortium': extractTagSheetTagItem(name,"consortium"),
            'study_accession': attributes.get("study_accession","Unspecified"),
            'tag:tags:dbGaP': extractTag(w,"dbGaP:"),
             #Extract dbGap tag item   
            'tagsSheet:tag:tags:dbGaP':  extractTagSheetTagItem(name,"dbGaP"),
            'library:dataUseRestriction': attributes.get("library:dataUseRestriction","Unspecified"),
            'tag:tags:consent_code': extractTag(w,"consent_code:"),
            'library:indication': attributes.get("library:indication","Unspecified"),
            'library:studyDesign': attributes.get("library:studyDesign","Unspecified"),
            'library:datatype.items': extractDataTypes(w),
            'library:numSubjects': attributes.get("library:numSubjects","Unspecified"),
            'subjectCount': entityCounts['subject'],
            'bucketName': bucketName,
            'sampleCount': entityCounts['sample'],
            'participantCount': entityCounts['participant'],
            'familyCount': entityCounts['family'],
            'discoveryCount': entityCounts['discovery'],
            'bucketSize': getBucketSize(bucketName),
            'requestorPays': getRequestorPays(bucketName),
            
            
        }
        catalogArray.append(wsVals)

catalogDF = pd.DataFrame(catalogArray)
catalogDF.set_index("name", inplace=True)
print(catalogDF)

fileName = 'AnVILCatalogWorkspacesPublicMetadata-'+datetime.today().strftime('%Y-%m-%d')+".csv"

catalogDF.to_csv(fileName)
          
!gsutil cp ./$fileName $WORKSPACE_BUCKET/notebooks    

1 - AnVIL_CMG_Broad_Muscle_KNC_WGS
2 - anvil_ccdg_broad_ai_ibd_daly_niddk_cho_wes
3 - AnVIL_CCDG_Broad_NP_Epilepsy_USABLC_GRU-NPU_WES
4 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAUBG_DS_EPI_NPU_MDS_WES
5 - AnVIL_CMG_BaylorHopkins_HMB-NPU_WES
6 - AnVIL_eMERGE_HMB-MDS_Imputed_GWAS
7 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAUMC_DS_NEURO_MDS_GSA-MD
8 - ANVIL_CMG_UWASH_DS-HFA
9 - AnVIL_eMERGE_DS-DEM_Imputed_GWAS
10 - AnVIL_CCDG_Broad_NP_Epilepsy_USAMGH_MGBB_HMB_MDS_WES
11 - AnVIL_CCDG_Broad_NP_Epilepsy_USACRW_EPI_ASZ_MED_MDS_WES
12 - AnVIL_eMERGE_GRU_Imputed_GWAS
13 - ANVIL_CMG_Broad_Muscle_Laing_WES
14 - AnVIL_eMERGE_NUgene_GRU_WGS
15 - AnVIL_CCDG_WashU_CVD_EOCAD_BioMe_WGS
16 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAICB_HMB-NPU-MDS_GSA-MD
17 - AnVIL_CSER_CHARM_GRU
18 - AnVIL_ccdg_asc_ndd_daly_talkowski_chung_asd_exome
19 - AnVIL_CCDG_Broad_NP_Epilepsy_DEUPUM_HMB-MDS_GSA-MD
20 - AnVIL_CCDG_Broad_CVD_EOCAD_TaiChi_WGS
21 - AnVIL_CCDG_WashU_CVD_Eufam_WGS
22 - AnVIL_CCDG_Broad_AI_IBD_Brant_DS-IBD_WGS
23 - AnVIL_ccdg_

177 - AnVIL_CCDG_Broad_NP_Epilepsy_USACCH_DS_NEURO_MDS_WES
178 - AnVIL_CCDG_Broad_NP_Epilepsy_DEUUTB_HMB-NPU_MDS_GSA-MD
179 - AnVIL_ccdg_asc_ndd_daly_talkowski_menashe_asd_exome
180 - AnVIL_CCDG_Broad_NP_Epilepsy_HKOSB_GRU_GSA-MD
181 - AnVIL_ccdg_asc_ndd_daly_talkowski_AGRE_asd_exome
182 - AnVIL_CMG_Broad_Orphan_Sweetser_WES
183 - AnVIL_CCDG_Broad_NP_Epilepsy_GBRUCL_DS_EARET_MDS_GSA-MD
184 - AnVIL_CCDG_Broad_NP_Epilepsy_GHAKNT_GRU_WES
185 - AnVIL_ccdg_asc_ndd_daly_talkowski_domenici_asd_exome
186 - AnVIL_CMG_Broad_Brain_Walsh_WES
187 - anvil_ccdg_broad_ai_ibd_daly_rioux_niddk_wes
188 - AnVIL_CMG_Broad_Rare_RGP_WES
189 - 1000G-high-coverage-2019
190 - AnVIL_CCDG_Baylor_CVD_HemStroke_ERICH_WGS
191 - AnVIL_CCDG_NYGC_NP_Alz_EFIGA_WGS
192 - anvil_ccdg_broad_ai_ibd_daly_kupcinskas_wes
193 - AnVIL_CCDG_Broad_NP_Epilepsy_USACHP_GRU_GSA-MD
194 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAIGI_GRU_WES
195 - AnVIL_CSER_NCGENES2_GRU
196 - AnVIL_CMH_GAFK_WGBS
197 - AnVIL_CSER_KidsCanSeq_GRU
198 - AnVIL_CCDG_Ba

343 - AnVIL_CCDG_Broad_NP_Epilepsy_KENKIL_GRU_WES
344 - anvil_ccdg_broad_ai_ibd_daly_moayyedi_imagine_wes
345 - anvil_ccdg_broad_ai_ibd_daly_rioux_igenomed_gsa
346 - AnVIL_CCDG_Broad_NP_Epilepsy_GHAKNT_GRU_GSA-MD
347 - AnVIL_CCDG_NYGC_NP_Autism_SEARCHLIGHT_DS_WGS
348 - AnVIL_CCDG_Broad_CVD_AFib_UCSF_WGS
349 - AnVIL_CCDG_Broad_NP_Epilepsy_USAEGP_GRU_WES
350 - AnVIL_CMG_Broad_Brain_Engle_WGS
351 - AnVIL_CCDG_Baylor_CVD_TexGen_DS_WGS
352 - AnVIL_CCDG_Broad_AI_IBD_McCauley_WGS
353 - AnVIL_CCDG_Broad_CVD_AF_TMDU_Cases_Arrays
354 - AnVIL_ccdg_asc_ndd_daly_talkowski_pericak-vance_asd_exome_
355 - AnVIL_ccdg_asc_ndd_daly_talkowski_weiss_asd_exome
356 - anvil_ccdg_broad_ai_ibd_daly_lira_share_wes
357 - AnVIL_CMG_Broad_Heart_PCGC-Tristani_WGS
358 - AnVIL_ccdg_asc_ndd_daly_talkowski_pericak-vance_asd_wgs
359 - anvil_ccdg_broad_daly_igsr_1kg_twist_wes
360 - AnVIL_CMG_Broad_Orphan_Lerner-Ellis_WGS
361 - AnVIL_CCDG_Broad_AI_IBD_Kugathasan_WGS
362 - anvil_ccdg_broad_ai_ibd_daly_alm_gmc_gsa
363 - AnVI

512 - ANVIL_CMG_BROAD_BRAIN_SHERR_WGS
513 - AnVIL_CCDG_Broad_NP_Epilepsy_ITAICB_HMB_NPU_MDS_WES
514 - anvil_ccdg_broad_ai_ibd_daly_rioux_niddk_gsa
515 - AnVIL_GTEx_V8_hg38
516 - AnVIL_CCDG_Baylor_CVD_HemStroke_Regards_DS_WGS
                                                   sheet:consortium  \
name                                                                  
AnVIL_CMG_Broad_Muscle_KNC_WGS                                  CMG   
anvil_ccdg_broad_ai_ibd_daly_niddk_cho_wes                     CCDG   
AnVIL_CCDG_Broad_NP_Epilepsy_USABLC_GRU-NPU_WES                CCDG   
AnVIL_CCDG_Broad_NP_Epilepsy_ITAUBG_DS_EPI_NPU_...             CCDG   
AnVIL_CMG_BaylorHopkins_HMB-NPU_WES                             CMG   
...                                                             ...   
ANVIL_CMG_BROAD_BRAIN_SHERR_WGS                                 CMG   
AnVIL_CCDG_Broad_NP_Epilepsy_ITAICB_HMB_NPU_MDS...             CCDG   
anvil_ccdg_broad_ai_ibd_daly_rioux_niddk_gsa                   CC

Copying file://./AnVILCatalogWorkspacesPublicMetadata-2022-02-25.csv [Content-Type=text/csv]...
/ [1 files][109.3 KiB/109.3 KiB]                                                
Operation completed over 1 objects/109.3 KiB.                                    


In [72]:
pp.pprint(getEntityCounts('AnVIL_CCDG_Broad_CVD_AFib_AFLMU_WGS'))

{   'discoveryCount': 0,
    'familyCount': 0,
    'participantCount': 248,
    'sampleCount': 248,
    'subjectCount': 0}


In [66]:
pp.pprint( FAPI.list_entity_types(CATALOG_WORKSPACE_NAMESPACE, 'AnVIL_CCDG_Broad_CVD_AFib_AFLMU_WGS').json())

{   'participant': {   'attributeNames': [   'collaborator_participant_id',
                                             'gender',
                                             'participant'],
                       'count': 248,
                       'idName': 'participant_id'},
    'qc_result_sample': {   'attributeNames': [   'cram',
                                                  'freemix',
                                                  'mean_coverage',
                                                  'median_absolute_deviation',
                                                  'median_insert_size',
                                                  'pct_10x',
                                                  'pct_20x',
                                                  'pct_30x',
                                                  'pct_chimeras',
                                                  'percent_duplication',
                                                  'q20_bases